In [ ]:
from pulp import *

def SolveLP(df, SquadComposition, MaxElementsPerTeam, BudgetLimit):

      # Get a list of players
    players = list(df['name'])
    # Initialize Dictionaries for Salaries and Positions
    salaries = dict(zip(players, df['value']))
    positions = dict(zip(players, df['position']))
    teams=dict(zip(players, df['team']))
    # Dictionary for Projected Score for each player
    project_points = dict(zip(players, df['total_points']))
    # Set Players to Take either 1 or 0 values (owned or not)
    player_vars = LpVariable.dicts("Player", players, lowBound=0, upBound=1, cat='Integer')

    total_score = LpProblem("FPL Best Team", LpMaximize)
    total_score += lpSum([project_points[i] * player_vars[i] for i in player_vars])
    total_score += lpSum([salaries[i] * player_vars[i] for i in player_vars]) <= BudgetLimit
    # Get indices of players for each position
    fwd = [p for p in positions.keys() if positions[p] == 'FWD']
    defD = [p for p in positions.keys() if positions[p] == 'DEF']
    mid = [p for p in positions.keys() if positions[p] == 'MID']
    gk = [p for p in positions.keys() if positions[p] == 'GK']
    # Set Constraints
    total_score += lpSum([player_vars[i] for i in fwd]) == SquadComposition["Forwards"]
    total_score += lpSum([player_vars[i] for i in defD]) == SquadComposition["Defenders"]
    total_score += lpSum([player_vars[i] for i in mid]) == SquadComposition["Midfielders"]
    total_score += lpSum([player_vars[i] for i in gk]) == SquadComposition["Goalkeepers"]


    # Teams constraints
    for k in list(df["team"].unique()):
        teamTMP=[p for p in teams.keys() if teams[p] == k]
        total_score += lpSum([player_vars[i] for i in teamTMP]) <= MaxElementsPerTeam

    total_score.solve()

    playersTeam=[]
    for v in total_score.variables():
        if v.varValue > 0:
            playersTeam.append(v.name.replace("Player_r_","").replace("_", " ").replace("Player ",""))
            print(v.name.replace("Player_r_","").replace("_", " ").replace("Player ",""))

    dfPlayers=pd.DataFrame(playersTeam)
    dfPlayers.columns=["name"]

    return (dfPlayers.merge(df, how="left"))